In [1]:
import nbformat as nbf
import glob
import json
import pandas as pd

In [2]:
all_logs = glob.glob("../logs/*.json")

# formats all logs properly 
# key = session ID
# value = json w/ same format as what was originally held in nb metadata
formatted_logs = {}

for log in all_logs:
    f = open(log, 'r')
    
    # merges body sent from various HTTP Posts
    entries = f.read().split('}{')
    for i in range(len(entries)):
        entries[i] = '{' + entries[i] + '}'
    entries[0] = entries[0][1:]
    entries[-1] = entries[-1][:-1]
    formatted_log = json.loads(entries[0])
    for i in range(1,len(entries)):
        j = json.loads(entries[i])
        history = j['history']
        formatted_log['history'].extend(history)
    
    # get time user started ipynb
    formatted_log['startTime'] = formatted_log['history'][0]['time']
    formatted_log['endTime'] = formatted_log['history'][len(formatted_log['history'])-1]['time']
    
    # separates user hash from file name (sessionID)
    names = log.split('_')
    formatted_log['userHash'] = names[0][5:]
    
    # excluding Jerry and Doris and potentially others
    exclude_list = ['bca87887a1cc89312f7d073fd007ea68', '1a735d0ee6a6f9d7fdab573b50851da7']
    if names[0][5:] not in exclude_list:
        formatted_logs[names[1][:-5]] = formatted_log


In [3]:
df = pd.DataFrame(formatted_logs).transpose().sort_values('userHash')
df

,nbName,history,startTime,endTime,userHash
6b6nhxtxh,cars_demo.ipynb,"[{'type': 'executeCodeCell', 'time': '2020-12-...",2020-12-27T01:20:56.927Z,2020-12-27T01:47:49.232Z,gs/003724cc5e41a981d3fad0ca3f21888d
czxswqqjy,1-specifying-intent.ipynb,"[{'type': 'executeCodeCell', 'time': '2020-12-...",2020-12-27T02:41:41.804Z,2020-12-27T03:01:14.187Z,gs/003724cc5e41a981d3fad0ca3f21888d
8pzgwyy22,1-specifying-intent.ipynb,"[{'type': 'executeCodeCell', 'time': '2020-12-...",2020-12-27T03:02:30.953Z,2020-12-27T03:12:26.956Z,gs/003724cc5e41a981d3fad0ca3f21888d
ug7hr39o8,cars_demo.ipynb,"[{'type': 'executeCodeCell', 'time': '2020-11-...",2020-11-30T05:36:43.016Z,2020-11-30T06:11:08.975Z,gs/003a65e7008e9f7db64a6559970f962c
97vczg8cn,cars_demo.ipynb,"[{'type': 'executeCodeCell', 'time': '2020-12-...",2020-12-03T02:55:46.498Z,2020-12-03T03:12:19.732Z,gs/006b045214badd8a59f7f11c1eae488c
...,...,...,...,...,...
ak5h0c64n,4-Data-Playground.ipynb,"[{'type': 'executeCodeCell', 'time': '2020-11-...",2020-11-04T22:20:55.715Z,2020-11-04T22:53:09.488Z,gs/fc836b6c537cc69b6eb6ab9cb7cb6b05
f1ukitdr9,1-Specify-Intent.ipynb,"[{'type': 'executeCodeCell', 'time': '2020-11-...",2020-11-04T21:34:36.099Z,2020-11-04T21:47:49.607Z,gs/fc836b6c537cc69b6eb6ab9cb7cb6b05
o995ki32j,cars_demo.ipynb,"[{'type': 'executeCodeCell', 'time': '2020-12-...",2020-12-01T06:17:35.472Z,2020-12-01T06:34:09.014Z,gs/fe50a4ea1e132d4d505b7941f1f48ef0
jiklvm3yu,cars_demo.ipynb,"[{'type': 'executeCodeCell', 'time': '2020-12-...",2020-12-25T16:37:15.015Z,2020-12-25T17:06:12.739Z,gs/feba0e7c024be16c21b46a6257a10367


In [4]:
len(df[df["nbName"]=='4-Data-Playground.ipynb'])

24

In [5]:
df["length"] = df.history.apply(lambda x: len(x))

In [6]:
# df[(df["nbName"]=='4-Data-Playground.ipynb') &(df["length"]>3)]

In [16]:
for entry in df.values:
    nb = nbf.v4.new_notebook()
    code = "import lux\nimport pandas as pd"
    nb['cells'] = [nbf.v4.new_code_cell(code)]
    nb['kernelspec'] = {"display_name": "Python 3", "language": "python","name": "python3"}
    for log_entry in entry[1]:
        if log_entry['type'] == 'executeCodeCell':
            nb['cells'].append(nbf.v4.new_code_cell(log_entry['code']))
    name = entry[4][3:] + entry[0]
    nbf.write(nb, '../constructed_nbs/' + name+".ipynb")